# [Probability of a Two Boxes Having The Same Number of Distinct Balls](https://leetcode.com/problems/probability-of-a-two-boxes-having-the-same-number-of-distinct-balls/)
- **Given**: `2N` balls with `K` distinct colors in
    + Array `balls[color]`: number of ball in each color, **size = K**, **sum = 2\*N**
    - Distribute all balls into 2 boxes: **box1** and **box2**; such that
        + number of balls in box1 = number of balls in box2 = `N`
- **Return**: The probability that **box1** and **box2** has the `same number of distinct colors`

#### Constraints
- $1 \leq K \leq 8$
- $1 \leq \text{balls[i]} \leq 6$
- Total number of balls = even

#### Example 1

```
Input: balls = [1,1]
Output: 1.00000
Explanation: P = 2/2
    [0 / 1] x
    [1 / 0] x
```

#### Example 2

```
Input: balls = [2,1,1]
Output: 0.66667
Explanation: P = 8 /12
    [0,0 / 1,2]
    [0,0 / 2,1]
    [0,1 / 0,2] x
    [0,1 / 2,0] x
    [0,2 / 0,1] x
    [0,2 / 1,0] x
    [1,0 / 0,2] x
    [1,0 / 2,0] x
    [1,2 / 0,0]
    [2,0 / 0,1] x
    [2,0 / 1,0] x
    [2,1 / 0,0]
```

#### Example 3

```
Input: balls = [1,2,1,2]
Output: 0.60000
Explanation: P = 108 / 180
```

## Solution
- **Multinomial coefficients**: The number of ways we can divide `n` elements into `m` subsets of sizes $k_1,k_2,...,k_m$
$$C^{k_1,k_2,...,k_m}_n = C^{k_1}_nC^{k_2}_{n-k_1}...C^{k_m}_{n-k_1-...k_{m-1}} = \frac{n!}{k_1!k_2!...k_m!}$$
- Compute by 3 methods
    - 1. factorial: $\frac{n!}{k_1!k_2!...k_m!}$
    - 2. combination:  $C^{k_1}_nC^{k_2}_{n-k_1}...C^{k_m}_{n-k_1-...k_{m-1}}$
    - 3. Running N

#### 1. Backtrack - factorial

```Cpp
class Solution {
public:
    // Combinatorics functions
    unordered_map<long long, double> fac;
    double factorial(long long x) {
        if(fac.count(x)) return fac[x];
        if(x == 0 || x == 1) return fac[x] = 1.0;
        return fac[x] = 1.0*x*factorial(x-1);
    }

    int N, K;
    vector<int> A;
    double fav_outcomes, sample_space;
    void dfs(int k, vector<int> box_1, int num_1, int color_1, vector<int> box_2, int num_2, int color_2) {
        // Base cases + prunes
        if(num_1 > N || num_2 > N) return;
        if(k == K && num_1 == num_2 && num_1 == N) {
            // Compute multinomial by factorial
            double config_1 = factorial(num_1);
            for(int b:box_1) config_1 /= factorial(b);

            double config_2 = factorial(num_2);
            for(int b:box_2) config_2 /= factorial(b);

            if(color_1 == color_2) fav_outcomes += config_1 * config_2;
            sample_space += config_1 * config_2;
            return;
        }
        if(k == K) return;

        // For each color(k) add b to box 1, A[k] - b to box 2
        for(int b=0; b<=A[k]; ++b) {
            box_1[k] = b;
            box_2[k] = A[k] - b;

            if(b > 0) color_1 += 1; 
            if(A[k] - b > 0) color_2 += 1;
            dfs(k+1, box_1, num_1 + b, color_1, box_2, num_2 + A[k]-b, color_2);
            if(b > 0) color_1 -= 1; 
            if(A[k] - b > 0) color_2 -= 1;
        }
    }
    // main
    double getProbability(vector<int> &balls) {
        A = balls;

        // N: total balls each bag
        N = accumulate(balls.begin(), balls.end(), 0);
        N /= 2;

        // K: total colors
        K = balls.size();

        fav_outcomes = sample_space = 0.0;
        vector<int> box_1(K, 0), box_2(K, 0);
        dfs(0, box_1, 0, 0, box_2, 0, 0);
        return fav_outcomes / sample_space;
    }
};
```

#### 2. DP - comb

```Cpp
class Solution {
public:
    // Combinatorics functions
    unordered_map<long long, double> fac;
    double factorial(long long x) {
        if(fac.count(x)) return fac[x];
        if(x == 0 || x == 1) return fac[x] = 1.0;
        return fac[x] = 1.0*x*factorial(x-1);
    }
    double comb(long long n, long long k) {
        return factorial(n) / (factorial(k) * factorial(n-k));
    }

    // Hash functions
    string str_hash(const vector<int> &keys) {
        string val = "";
        for(const int &k: keys) val += to_string(k) + ',';
        return val + 'S';
    }
    vector<int> rev_hash(const string &key) {
        vector<int> res;

        string tmp = "";
        for(const char &c:key) {
            if(c == 'S') break;
            if(c == ',') {
                res.push_back(stoll(tmp));
                tmp = "";
            } else tmp += c;
        }
        return res;
    }

    // main
    double getProbability(vector<int> &balls) {
        vector<int> A = balls;

        // N: total balls each bag
        int N = accumulate(balls.begin(), balls.end(), 0);
        N /= 2;

        // K: total colors
        int K = balls.size();

        // dp[{total_ball_1, color_size_1, total_ball_2, color_size_2}] = total cases
        unordered_map<string, double> dp, dp_next;
        dp[str_hash({0,0,0,0})] = 1.0;

        // For each color(k) add b to box 1, A[k] - b to box 2
        double sample_space = 0.0, fav_outcomes = 0.0;
        for(int k=0; k<K; ++k) {
            for(int b=0; b<=A[k]; ++b) {
                for(auto [key, val]: dp) {
                    auto keys = rev_hash(key);
                    int num_1 = keys[0];
                    int color_1 = keys[1];
                    int num_2 = keys[2];
                    int color_2 = keys[3];

                    int color_added_1, color_added_2;
                    double config_1, config_2;

                    // Add b to box 1: Compute multinomial by combination
                    if(N-num_1 < b) continue;
                    if(b == 0) {
                        color_added_1 = 0;
                        config_1 = 1.0;
                    } else {
                        color_added_1 = 1;
                        config_1 = comb(N-num_1, b);
                    }

                    // Add A[k] - b to box 2: Compute multinomial by combination
                    if(N-num_2 < A[k] - b) continue;
                    if(A[k] - b == 0) {
                        color_added_2 = 0;
                        config_2 = 1.0;
                    } else {
                        color_added_2 = 1;
                        config_2 = comb(N-num_2, A[k] - b);
                    }

                    // update dp
                    num_1 += b;
                    color_1 += color_added_1;
                    num_2 += A[k] - b;
                    color_2 += color_added_2;
                    if(num_1 <= N && num_2 <= N) dp_next[str_hash({num_1,color_1,num_2,color_2})] += dp[key] * config_1 * config_2;

                    // Relax ans
                    if(num_1 == num_2 && num_1 == N) {
                        if(color_1 == color_2) fav_outcomes += dp[key] * config_1 * config_2;
                        sample_space += dp[key] * config_1 * config_2;
                    }
                }
            }
            dp = dp_next;
        }
        return fav_outcomes / sample_space;
    }
};
```

#### 3. DP - Running N_

```Cpp
class Solution {
public:
    // Combinatorics functions
    unordered_map<long long, double> fac;
    double factorial(long long x) {
        if(fac.count(x)) return fac[x];
        if(x == 0 || x == 1) return fac[x] = 1.0;
        return fac[x] = 1.0*x*factorial(x-1);
    }

    // Hash functions
    string str_hash(const vector<int> &keys) {
        string val = "";
        for(const int &k: keys) val += to_string(k) + ',';
        return val + 'S';
    }
    vector<int> rev_hash(const string &key) {
        vector<int> res;

        string tmp = "";
        for(const char &c:key) {
            if(c == 'S') break;
            if(c == ',') {
                res.push_back(stoll(tmp));
                tmp = "";
            } else tmp += c;
        }
        return res;
    }

    // main
    double getProbability(vector<int> &balls) {
        vector<int> A = balls;

        // N: total balls each bag
        int N = accumulate(balls.begin(), balls.end(), 0);
        N /= 2;

        // K: total colors
        int K = balls.size();

        // dp[{total_ball_1, color_size_1, total_ball_2, color_size_2}] = total cases
        unordered_map<string, double> dp, dp_next;
        dp[str_hash({0,0,0,0})] = 1.0;

        // For each color(k) add b to box 1, A[k] - b to box 2
        double sample_space = 0.0, fav_outcomes = 0.0;
        for(int k=0; k<K; ++k) {
            for(int b=0; b<=A[k]; ++b) {
                for(auto [key, val]: dp) {
                    auto keys = rev_hash(key);
                    int num_1 = keys[0];
                    int color_1 = keys[1];
                    int num_2 = keys[2];
                    int color_2 = keys[3];

                    int color_added_1, color_added_2;
                    double config_1, config_2;

                    // Add b to box 1: Compute multinomial by running N
                    if(b == 0) {
                        color_added_1 = 0;
                        config_1 = 1.0;
                    } else {
                        color_added_1 = 1;
                        config_1 = factorial(num_1 + b) / (factorial(num_1) * factorial(b));
                    }

                    // Add A[k] - b to box 2: Compute multinomial by running N
                    if(A[k] - b == 0) {
                        color_added_2 = 0;
                        config_2 = 1.0;
                    } else {
                        color_added_2 = 1;
                        config_2 = factorial(num_2 + A[k] - b) / (factorial(num_2) * factorial(A[k] - b));
                    }

                    // update dp
                    num_1 += b;
                    color_1 += color_added_1;
                    num_2 += A[k] - b;
                    color_2 += color_added_2;
                    if(num_1 <= N && num_2 <= N) dp_next[str_hash({num_1,color_1,num_2,color_2})] += dp[key] * config_1 * config_2;

                    // Relax ans
                    if(num_1 == num_2 && num_1 == N) {
                        if(color_1 == color_2) fav_outcomes += dp[key] * config_1 * config_2;
                        sample_space += dp[key] * config_1 * config_2;
                    }
                }
            }
            dp = dp_next;
        }
        return fav_outcomes / sample_space;
    }
};
```